Today we will talk about optimization.
We will focus on unconstrained optimization in one dimension.

To fix ideas we will deal with minimization (unless stated otherwise):

$$\min_x f(x)$$


In [ ]:
using Pkg
Pkg.activate("..") ## because we have environment files in the parent directory
Pkg.instantiate() ## to download all missing packages


In [ ]:
# load some packages we will need today
using Plots, Calculus, Optim, ForwardDiff

### Grid search
Just evaluate the function on some grid and pick the point that has the largest/the smallest value.

In [ ]:
# test function
f(x) = x.^2.0 .+ 2.11 .* x .+ 0.98  #notice the dots! 

# plot it to see what happens
plot(f, -5, 5, label="f(x)", legend=:topleft, title = "our amazing function")

In [ ]:
grid_for_evaluation = -5:0.1:5 # this is the grid we will use to evaluate the function
typeof(grid_for_evaluation) # what is the type of this thing?

In [ ]:
vector_of_points = collect(grid_for_evaluation) # convert the range to a vector -- not needed for much, but you might want to extract the point

In [ ]:
# evaluate

function_values = f(grid_for_evaluation) # evaluate the function on the grid

In [ ]:
val_min, arg_min_ind = findmin(function_values)
arg_min = grid_for_evaluation[arg_min_ind] 


# more compactly
arg_min_2 = argmin(f, grid_for_evaluation) 

println("The minimum is at (?) $arg_min with value $val_min")

### Bracketing methods
As you can see, the above is not super precise. We can add more and more grid points, but there are better methods. If we know an optimum lies in some interval, we can use bracketing methods.

In [ ]:
function simple_bracket(f,a,b,c; ε=10e-6, maxcounter = 100, verbose = false)
    # this algorithm is from Judd (1988), page 95

    counter = 1
    if (f(a) > f(b) && f(c) > f(b)) && (a < b < c)
        

        guesses = []
        # try with a new point d 
        while c - a > ε

            if counter > maxcounter
                println("Maximum number of iterations ($maxcounter) reached")
                break
            end

            if (b - a < c - b)
                d = (b+c)/2
            else
                d = (a+b)/2
            end
                
            guesses = push!(guesses,d)
            
            if verbose
                println("Iteration = $counter")    
                println("Point d = [$d]")    
                println("")
            end


            counter += 1
            if ((d < b) && f(d) > f(b))
                a = d
                b = b
                c = c
            elseif ((d < b) && f(d) < f(b))
                a = a
                b = d
                c = b
            elseif ((d > b) && f(d) < f(b))
                a = b
                b = d
                c = c
            else
                a = a
                b = b
                c = d
            end

            
        
        end
            println("Minimum = $(f(c)) at $c")
            return (argmin = c, val = f(c), points = guesses, iteration = counter)

        
        
    else 
        println("The condition a < b < c and f(a), f(c) > f(b) is not satisfied. Try with different values.")
    end
end

In [ ]:
outs = simple_bracket(f,-4,0,4;verbose=true)

In [ ]:
outs

In [ ]:
myscatter = scatter(;xlim = [-5,0],ylim = [0,3])
animation = @animate for (ind,point) in enumerate(outs.points)
    scatter!(myscatter,[point], [f(point)], alpha = 1 - ind/length(outs.points),ms = 5,color = :blue,lab="")
end

gif(animation,fps = 10)

In [ ]:
function newton(f,f′,f′′,x0; ε=10e-6, δ=10e-6, maxcounter = 100, verbose = false)
    # this algorithm is from Judd (1988), page 98
    x_old = x0
    x_new = 2*abs(x0) + 1
    counter = 1
    guesses = []

    while ((abs(f′(x_old)) > δ) || (abs(x_new-x_old) > ε * (1+abs(x_old))))
        guesses = push!(guesses,x_old)
        if verbose
            println("Iteration = $counter")    
            println("Point = $x_old")    
            println("Value = $(f(x_old))")  
            println("Derivative = $(f′(x_old))")  
            println("")
        end


        if counter > maxcounter
            println("Maximum number of iterations ($maxcounter) reached")
            break
        end

        counter += 1
        x_old = x_new
        x_new = x_old - f′(x_old)/f′′(x_old)
        
        
    end

    return (argmin = x_new, val = f(x_new), derivative  = f′(x_new), points = guesses, iteration = counter)
    end

In [ ]:
#redefine function
f(x) = x.^2.0 .+ 2.11 .* x .+ 0.98  #notice the dots! 
f′(x) = 2.0 .* x .+ 2.11
f′′(x) = 2.0 

In [ ]:
newton(f,f′,f′′, 0.0,verbose=true)

In [ ]:

# use Calculus.jl package to get them - finite differences
# finite differences use the fact that the derivative is the limit of the difference quotient
# f′(x) = lim_{h->0} (f(x+h) - f(x))/h
# so we can approximate it with a small h

f′(x) = derivative(f,x)
f′′(x) = second_derivative(f,x)

plot(f, -5, 5, label="f(x)", legend=:topleft, title = "our amazing function")
plot!(f′, -5, 5, label="f′(x)", legend=:topleft)
plot!(f′′, -5, 5, label="f′′(x)", legend=:topleft)


In [ ]:
# automatic differentiation - it is exact
# ForwardDiff.jl package
ForwardDiff.derivative(f,1) # first derivative at 1
f′_AD(x) = ForwardDiff.derivative(f,x) # function that returns the derivative of f at x - note it takes f defined earlier



In [ ]:
function newton_AD(f,x0; ε=10e-6, δ=10e-6, maxcounter = 100, verbose = false)
    # this algorithm is from Judd (1988), page 98
    f′(x)  = ForwardDiff.derivative(f,x)
    f′′(x) = ForwardDiff.derivative(f′,x)

    x_old = x0
    x_new = 2*abs(x0) + 1
    counter = 1
    guesses = []

    while ((abs(f′(x_old)) > δ) || (abs(x_new-x_old) > ε * (1+abs(x_old))))
        guesses = push!(guesses,x_old)
        if verbose
            println("Iteration = $counter")    
            println("Point = $x_old")    
            println("Value = $(f(x_old))")  
            println("Derivative = $(f′(x_old))")  
            println("")
        end


        if counter > maxcounter
            println("Maximum number of iterations ($maxcounter) reached")
            break
        end

        counter += 1
        x_old = x_new
        x_new = x_old - f′(x_old)/f′′(x_old)
        
        
    end

    return (argmin = x_new, val = f(x_new), derivative  = f′(x_new), points = guesses, iteration = counter)
    end

In [ ]:
newton_AD(f, 0.0,verbose=true)